In [1]:
# ploting libraries
import matplotlib.pyplot as plt
%matplotlib inline

# import PreProcessing functions
import sys 
import os
sys.path.append(os.path.abspath("C:\\Users\\koriv\\Desktop\\MachineLearning_DataScience\\Hands_On_Machine_Learning\\my_env_codebasics\\My_CODE\\ML-projects\\advanced_House_Price_Prediction-Regression\\preProcessingObjects_list"))
from advanced_House_Price_Prediction_PreProcessing import *


# deplyment libraries
import pickle

# Importing required framework libraries
import pandas as pd
import numpy as np

# Import required pipeline and transformation libraries
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Import processing and evaluation libraries
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.model_selection import GridSearchCV

# Regressors for Data
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor

# Regressors for Imputer
from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor

## for feature slection
from sklearn.feature_selection import SelectFromModel

## for metrics
from sklearn.metrics import explained_variance_score
from sklearn.metrics import max_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

# Import UnProcessed Test features

In [2]:
#Import testing features
import pathlib
path_to_read_model = 'C:\\Users\\koriv\\Desktop\\MachineLearning_DataScience\\Hands_On_Machine_Learning\\my_env_codebasics\\My_CODE\\ML-projects\\advanced_House_Price_Prediction-Regression\\probleam_study' #Path of current working Directory
with open(path_to_read_model + '\\testFeatures_list.pkl', 'rb') as f:
    X_test, y_test = pickle.load(f)

## Verify the match of the Processed Data column Names

In [4]:
#Import testing features
import pathlib
path_to_read_model = 'C:\\Users\\koriv\\Desktop\\MachineLearning_DataScience\\Hands_On_Machine_Learning\\my_env_codebasics\\My_CODE\\ML-projects\\advanced_House_Price_Prediction-Regression\\encoding_scaling_imputation_modelling' #Path of current working Directory
with open(path_to_read_model + '\\feature_ColumnNames.pkl', 'rb') as f:
    feature_ColumnNames = pickle.load(f)

In [5]:
X_test.columns == feature_ColumnNames

array([ True,  True,  True,  True,  True,  True,  True,  True])

# Data PreProcessing

In [6]:
# Define a function to apply the imported functions
def preProcessing(featureDF, targetDF, function_list):
    for function in function_list:
        featureDF, targetDF = function(featureDF, targetDF)
    return featureDF, targetDF

X_test, y_test = preProcessing(X_test, y_test, [Transformations, outlierRemoval])

column: size, lower_limit: 0.5, upper_limit: 4.5
column: total_sqft, lower_limit: 215.0, upper_limit: 2575.0
column: bath, lower_limit: 0.5, upper_limit: 4.5
column: balcony, lower_limit: -0.5, upper_limit: 3.5


## Encoding and Modeling

In [7]:
#Import testing features
import pathlib
path_to_read_model = 'C:\\Users\\koriv\\Desktop\\MachineLearning_DataScience\\Hands_On_Machine_Learning\\my_env_codebasics\\My_CODE\\ML-projects\\advanced_House_Price_Prediction-Regression\\encoding_scaling_imputation_modelling' #Path of current working Directory
with open(path_to_read_model + '\\estimators.pkl', 'rb') as f:
    ct_O, best_algos = pickle.load(f)

In [8]:
X_test[X_test.columns] = ct_O.transform(X_test)

In [9]:
best_algos

{'RandomForestRegressor': {'best_estimator': Pipeline(steps=[('ct_ohe_sca',
                   ColumnTransformer(transformers=[('pp_catN',
                                                    Pipeline(steps=[('col_catN',
                                                                     SimpleImputer(fill_value='missing',
                                                                                   strategy='constant')),
                                                                    ('catN',
                                                                     OneHotEncoder(handle_unknown='ignore',
                                                                                   sparse=False))]),
                                                    ['availability', 'location']),
                                                   ('pp_num',
                                                    Pipeline(steps=[('scaler',
                                                           

# Random Forest Regressor on Testing Data

In [10]:
# Get the object of the Best RandomForestRegressor Algorithm with the best params
be_RF = best_algos['RandomForestRegressor']['best_estimator']

In [11]:
be_RF.score(X_test, y_test) #Default is explained_variance_score

0.5067082394263884

## Model evaluation metrics RandomForestRegressor

In [13]:
y_testpred_RF = be_RF.predict(X_test)

In [14]:
## Mean of Predicted and True Targets
print("mean of Predicted values: ",np.mean(y_testpred_RF, axis = None))
print("mean of True values: ",np.mean(y_test, axis = None))

mean of Predicted values:  79.65768880641863
mean of True values:  79.49576286764704


In [18]:
## Using explained_variance_score 
explained_variance_score(y_test, y_testpred_RF)

0.5067184883175453

In [20]:
## Using max_error
max_error(y_test, y_testpred_RF)

329.36994760305834

In [21]:
## Using neg_root_mean_squared_error
mean_squared_error(y_test, y_testpred_RF, squared=False)

35.52465867403716

In [22]:
## Using R2(r2_score)
r2_score(y_test, y_testpred_RF)

0.5067082394263884

# Lasso Regressor on Training Data

In [23]:
# Get the object of the Best Lasso Algorithm with the best params
be_Lasso = best_algos['Lasso']['best_estimator']

In [24]:
# View the model coeeficients
be_Lasso.named_steps['model'].coef_
# Zero value in the coefficients indicates multicollnear columns, these columns can be eliminated in the future selection process.

array([-0.00000000e+00, -1.05038294e+00,  1.98027688e+00,  6.90292541e-01,
        0.00000000e+00,  0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
        8.77835065e+00, -2.23535901e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00,  3.09830665e+01, -0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,  8.29313394e+00,
        0.00000000e+00,  4.17657927e+00,  0.00000000e+00,  0.00000000e+00,
       -1.69868812e+00,  2.06472550e+01, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00,  0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
       -3.48057124e+00, -0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
       -0.00000000e+00, -1.76286642e+00, -1.34489433e+01, -0.00000000e+00,
        2.01665773e+01, -

In [25]:
# View the model intercept
be_Lasso.named_steps['model'].intercept_

11.615967376628333

## Model evaluation metrics Lasso Regressor

In [27]:
y_testpred_Lasso = be_RF.predict(X_test)

In [28]:
## Mean of Predicted and True Targets
print("mean of Predicted values: ",np.mean(y_testpred_Lasso, axis = None))
print("mean of True values: ",np.mean(y_test, axis = None))

mean of Predicted values:  79.65768880641863
mean of True values:  79.49576286764704


In [29]:
## Using explained_variance_score 
explained_variance_score(y_test, y_testpred_Lasso)

0.5067184883175453

In [30]:
## Using max_error
max_error(y_test, y_testpred_Lasso)

329.36994760305834

In [31]:
## Using neg_root_mean_squared_error
mean_squared_error(y_test, y_testpred_Lasso, squared=False)

35.52465867403716

In [32]:
## Using R2(r2_score)
r2_score(y_test, y_testpred_Lasso)

0.5067082394263884